# Lab | Random Forests

For this lab, you will be using the CSV files provided in the files_for_lab folder. These are cleaned versions of the learningSet data from the Case Study 'Healthcare for All'.

Instructions  
- Apply the Random Forests algorithm but this time only by upscaling the data.
- Use Feature Selections that you have learned in class to decide if you want to use all of the features (PCA, etc)
- Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?

## Importing Libraries

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

#train-test-split
from sklearn.model_selection import train_test_split

#categorical variables
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

#for regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

#for upsampling
from sklearn.utils import resample

#for pickling
import pickle

## Functions: 
- to check for null values in all the columns
- to check the data types

In [80]:
#Function to check for NaNs:
def get_percentnan(dataframe):
    data_percentnan = pd.DataFrame((dataframe.isna().sum()/len(dataframe))*100)
    data_percentnan = data_percentnan.reset_index()
    data_percentnan.columns = ['column_name', 'nulls_percentage']
    data_percentnan = data_percentnan[(data_percentnan['nulls_percentage'] > 0)]
    
    return data_percentnan

In [81]:
#check the columns and the data types
def data_colandtype(dataframe):
    data_col = pd.DataFrame(dataframe.iloc[0 , :])
    data_col['type'] = dataframe.dtypes
    unique_values = []
    for column in dataframe.columns:
        unique_values.append(len(dataframe[column].unique()))
    data_col['#_unique_values'] = unique_values
    data_col = data_col.reset_index()
    data_col.columns = ['header_name', 'row_0', 'type','#_unique_values']
    pd.set_option('display.max_rows', None)
    display(data_col)
    return


In [82]:
#plot categorical countplots
def plot_categorical_countplot(categorical_df):
    for column in categorical_df.columns:
        plt.figure(figsize=(5, 5))
        
        ax = sns.countplot(x=column, data=categorical_df)
        #ax = sns.histplot(data=categorical_df, x=column, stat="percent")#, multiple="dodge", shrink=.8)

        #rotating the ticklabels on the x-asis:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
        plt.tight_layout()
        plt.show()

## Get data and have a look

In [83]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
target = pd.read_csv('target.csv')

In [84]:
display(numerical.shape)
numerical.head()

(95412, 315)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

In [85]:
display(categorical.shape)
categorical.head()

(95412, 22)

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3


In [86]:
display(target.shape)
target.head()

(95412, 2)

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [87]:
data = pd.concat([numerical, categorical, target], axis = 1)

In [88]:
get_percentnan(data)

,column_name,nulls_percentage


## X/y-Split

In [89]:
y = target['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

In [90]:
# no need to scale the numericals or to encode the categoricals for the randomfores model
# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
# Scaling of the numericals is not important as well for DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
X = pd.concat([numericalX, encoded_categorical], axis = 1)

In [91]:
# we will use MinMaxScaler for the numericals since we will need it later to Features Selections
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numerical)

In [92]:
# Note: we need to do train/test split before upsampling, and then only upsampling the training set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Upsampling

In [93]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

In [94]:
category_0 = trainset[trainset['TARGET_B'] == 0]
display(category_0.shape)
category_1 = trainset[trainset['TARGET_B'] == 1]
display(category_1.shape)

(72486, 356)

(3843, 356)

In [95]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

display(category_1_oversampled.shape)

(72486, 356)

In [96]:
trainset_new = pd.concat([category_0, category_1_oversampled], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows

In [97]:
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(144972, 355)


In [98]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [99]:
y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

## RandomForestClassifier

In [100]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,           #max number of questions to ask
                             min_samples_split=20,  # ymount of rows still considered at every question
                             min_samples_leaf =20,  # ultimate answer based on at least this many rows
                             max_samples=0.8,       # fraction of X-train to use in each tree
                             random_state = 42)


In [101]:
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.6274384018982976
0.6052507467379343


In [102]:
y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11009,  7074],
       [  459,   541]])

### Cross-Validation

In [103]:
# For cross validation
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8
                             )
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))


0.6211406138771103


In [104]:
cross_val_score(clf, X_train, y_train, cv=5, scoring='recall_macro')

array([0.61869348, 0.61731293, 0.61588605, 0.62502587, 0.62164586])

## Feature Selections


### Numericals:

I use the variance threshold, because in the previous lab I used KBest.

### Variance threshold

In [105]:
pd.DataFrame(numerical_scaled).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314
0,0.000000,0.608247,0.666667,1.000000,0.000000,0.000000,0.393939,0.343434,0.181818,0.101010,0.020202,0.011494,0.555556,0.010051,0.011108,0.009378,0.0,0.353535,0.656566,0.474747,0.535354,0.929293,0.010101,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.113402,0.0,0.000000,0.000000,0.464286,0.571429,0.607143,0.476190,0.595238,0.642857,0.333333,0.313131,0.424242,0.272727,0.111111,0.141414,0.181818,0.171717,0.131313,0.111111,0.151515,0.121212,0.111111,0.343434,0.252525,0.181818,0.262626,0.101010,0.232323,0.181818,0.333333,0.494949,0.282828,0.121212,0.040404,0.616162,0.070707,0.164384,0.191919,0.304615,0.394286,0.979798,0.959596,0.020202,0.020202,0.000000,0.000000,0.070707,0.070707,0.000000,0.079833,0.105833,0.230769,0.153846,0.868687,0.141414,0.969697,0.040404,0.070707,0.383838,0.808081,0.707071,0.323232,0.848485,0.161616,0.060606,0.04,0.050505,0.090909,0.151515,0.030303,0.226667,0.505051,0.252525,0.000000,0.000000,0.000000,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.010101,0.616162,0.682353,0.677778,0.245902,0.100,0.020202,0.000000,0.000000,0.141414,0.010101,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.774120,0.204667,0.212000,0.232667,0.252000,0.073818,0.131313,0.232323,0.232323,0.232323,0.151515,0.010101,0.00,0.000000,0.010101,0.040404,0.252525,0.242424,0.262626,0.171717,0.020202,0.00,0.00000,0.020202,0.282828,0.040404,0.515152,0.010101,0.464646,0.545455,0.030303,0.888889,0.080808,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.040404,0.010101,0.131313,0.155556,0.210526,0.020202,0.454545,0.565657,0.646465,0.505051,0.646465,0.444444,0.626263,0.535354,1.000000,0.0,0.000000,0.090909,0.030303,0.080808,0.131313,0.090909,0.000000,0.054545,0.090909,0.030303,0.151515,0.191919,0.050505,0.040404,0.030303,0.0,0.030303,0.414141,0.010101,0.000000,0.070707,0.131313,0.060606,0.050505,0.000000,0.059701,0.090909,0.040404,0.010101,0.030303,0.101010,0.020202,0.010101,0.070707,0.787879,0.020202,0.000000,0.705882,0.161616,0.101010,0.393939,0.212121,0.216216,0.040404,0.030303,0.051546,0.202020,0.100000,0.263889,0.040404,0.000000,0.000000,0.000000,0.181818,0.393939,0.000000,0.343434,0.232323,0.181818,0.161616,0.012048,0.040404,0.000000,0.250000,0.000000,0.000000,0.050505,0.018182,0.000000,0.000000,0.000000,0.000000,0.00,0.074074,0.0,0.030303,0.747475,0.888889,0.080808,0.000000,0.040404,0.969697,0.777778,0.191919,0.419355,0.596154,0.050505,0.141414,0.141414,0.313131,0.545455,0.464646,0.0,0.0,0.909091,0.000000,0.101010,0.000000,0.0,0.000000,0.333333,0.656566,0.404040,1.000000,1.000000,0.285714,0.4,0.101010,0.070707,0.433333,0.366492,0.315789,0.168831,0.023965,0.127119,0.341463,0.005,0.001401,0.010,0.003676,0.006465,0.498045,0.0,1.000000,0.622951
1,0.000014,0.463918,0.833333,1.000000,0.066390,0.000000,0.151515,0.555556,0.111111,0.060606,0.020202,0.011494,1.000000

In [106]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.1
sel = VarianceThreshold(threshold=(var_threshold)) #(var_threshold) is the same for all the columns

# 1) default is threshold = 0: eliminates all columns that are identical for all rows
# 2) in practice we would scale the columns first, and then apply threshold, or apply 
#    a different threshold for different columns

# This drops the columns that have a variance less than this threshold
sel = sel.fit(numerical_scaled)
numerical_selected = sel.transform(numerical_scaled)
numerical_selected = pd.DataFrame(numerical_selected)
print(numerical_scaled.shape)
print(numerical_selected.shape)

(95412, 315)
(95412, 16)


In [107]:
pd.DataFrame(numerical_selected).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.555556,0.0,0.656566,0.070707,0.131313,0.272727,0.020202,0.050505,0.171717,0.0,0.313131,0.545455,0.909091,0.404040,0.0,1.000000
1,1.000000,1.0,0.000000,0.989899,0.989899,0.989899,0.919192,0.919192,0.919192,1.0,0.979798,1.000000,0.969697,1.000000,0.0,0.333333
2,0.111111,0.0,0.989899,0.010101,0.060606,0.181818,0.000000,0.020202,0.090909,1.0,0.505051,0.696970,0.000000,0.171717,1.0,1.000000
3,0.000000,0.0,0.929293,0.252525,0.505051,0.696970,0.010101,0.080808,0.171717,1.0,0.393939,0.454545,0.454545,0.222222,1.0,1.000000
4,1.000000,1.0,0.000000,0.010101,0.020202,0.161616,0.252525,0.585859,0.747475,1.0,0.060606,0.333333,0.090909,0.212121,1.0,0.333333


In [108]:
# To check which columns were removed, you can manipulate the results
sel.variances_

array([1.75493674e-04, 2.21932083e-02, 8.02275908e-02, 9.03764075e-02,
       1.49132173e-03, 2.61802339e-03, 1.34243682e-02, 2.32567811e-02,
       3.19975201e-02, 1.96784193e-03, 2.68155697e-03, 2.28104662e-03,
       1.02613600e-01, 3.38581196e-03, 3.76408939e-03, 3.59628980e-03,
       2.29533182e-01, 9.94600474e-02, 1.63556065e-01, 3.16042737e-03,
       3.39492297e-03, 4.50544144e-02, 2.84235197e-02, 1.22804261e-03,
       5.09957894e-03, 1.93912343e-02, 9.53325826e-04, 9.61133151e-04,
       6.55013636e-04, 1.18787514e-03, 4.75926813e-04, 5.03014665e-04,
       2.63911441e-04, 1.36515733e-02, 5.14425138e-04, 1.56917030e-03,
       1.53123652e-03, 9.84719265e-03, 9.64517465e-03, 9.31928327e-03,
       7.47231939e-03, 6.86524606e-03, 6.71358129e-03, 1.00407315e-02,
       6.91452469e-03, 4.15375273e-03, 3.46457167e-03, 3.75706547e-03,
       6.18802365e-03, 3.93599421e-03, 1.90088942e-03, 1.75339900e-03,
       3.67776591e-03, 4.61258144e-03, 2.67556157e-03, 1.39422327e-03,
      

In [109]:
#This gives you the result as True and False for the columns that we
# selected and those which were removed, respectively.
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())
var_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False

In [110]:
# list of columns that should be removed
removed_columns = [col[0] for col in zip(numerical.columns, var_list) if col[1] == False]
removed_columns

['TCODE',
 'AGE',
 'INCOME',
 'WEALTH1',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C2',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HU3',
 'HU4',
 'HU5',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 

In [111]:
len(removed_columns)

299

### X/y-Split (2)

In [112]:
y = target['TARGET_B']
X = pd.concat([numerical_selected, encoded_categorical], axis = 1)

In [113]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.555556,0.0,0.656566,0.070707,0.131313,0.272727,0.020202,0.050505,0.171717,0.0,0.313131,0.545455,0.909091,0.404040,0.0,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.000000,1.0,0.000000,0.989899,0.989899,0.989899,0.919192,0.919192,0.919192,1.0,0.979798,1.000000,0.969697,1.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.111111,0.0,0.989899,0.010101,0.060606,0.181818,0.000000,0.020202,0.090909,1.0,0.505051,0.696970,0.000000,0.171717,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.000000,0.0,0.929293,0.252525,0.505051,0.696970,0.010101,0.080808,0.171717,1.0,0.393939,0.454545,0.454545,0.222222,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.000000,1.0,0.000000,0.010101,0.020202,0.161616,0.252525,0.585859,0.747475,1.0,0.060606,0.333333,0.090909,0.212121,1.0,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [114]:
X.columns = X.columns.astype(str)

### Train/Test-Split (2)

In [115]:
# Note: we need to do train/test split before upsampling, and then only upsampling the training set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Upsampling (2)

In [116]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

In [117]:
category_0 = trainset[trainset['TARGET_B'] == 0]
display(category_0.shape)
category_1 = trainset[trainset['TARGET_B'] == 1]
display(category_1.shape)

(72486, 41)

(3843, 41)

In [118]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

display(category_1_oversampled.shape)

(72486, 41)

In [119]:
trainset_new = pd.concat([category_0, category_1_oversampled], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows

In [120]:
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(144972, 40)


In [121]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

### RandomForestClassifier (2)

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,           #max number of questions to ask
                             min_samples_split=20,  # ymount of rows still considered at every question
                             min_samples_leaf =20,  # ultimate answer based on at least this many rows
                             max_samples=0.8,       # fraction of X-train to use in each tree
                             random_state = 42)


In [123]:
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.5989225505614877
0.603626264214222


In [124]:
pickle.dump(clf,open('clf.p', 'wb'))

In [125]:
y_pred_test = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred_test))

0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[10970,  7113],
       [  451,   549]])

## Conclusion

In [126]:
# predicted         | False | True  |
# -----------------------------------
# actual  False     |   +   |       |
# -----------------------------------
#         True      |       |   +   |

We can see that our score has not really changed when considering all the features (315 columns for the numericals) or only the feature selection (16 columns for the numericals). It has even slightly worsened.
When comparing the confusion matrices we can see that the results have worsened as well: we predicted more donors as "true" donors than they actually were (7080 vs. 7340) and less false/false donnors (11003 vs. 10743). The predicted False / actual True and Predicted True / Actual True numbers did not differ much.

In this lab, I concentrated only on the feature selection for the numericals based on the variance threshold, because I did the feature selection already in the Lab 04.

Is the cost of a false positive equals to the cost of the false negative? 

No, a false positive, will only cost the postage/mail cost of sending the letter, whereas a false negative would mean that we would lose a donation, that is most probably going to be higher than the postage/mail cost. 

How would you change your algorithm or data in order to maximize the return of the business?  

I would have a closer look at the feature selection, as I wrote earlier, the feature selection for the categoricals wasn't done. Additionnaly we can with a model pipeline have a quick look at which model would give better results and with which parameters.

## Prepare for Lab 7_06 - Apply the model onto our data

In [127]:
final_y = pd.DataFrame(target['TARGET_B'])
final_data = pd.concat([numerical_selected, encoded_categorical], axis = 1)
display(final_y.shape)
display(final_data.shape)
final_y.head()


(95412, 1)

(95412, 40)

,TARGET_B
0,0
1,0
2,0
3,0
4,0


In [128]:
final_data.columns = final_data.columns.astype(str)

In [129]:
final_y['TARGET_B_Predicted'] = clf.predict(final_data)

In [130]:
final_y.shape

(95412, 2)

In [131]:
final_data_y_pred = pd.concat([final_data, final_y], axis = 1)

In [132]:
final_data_y_pred.shape
final_data_y_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B,TARGET_B_Predicted
0,0.555556,0.0,0.656566,0.070707,0.131313,0.272727,0.020202,0.050505,0.171717,0.0,0.313131,0.545455,0.909091,0.404040,0.0,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0,1
1,1.000000,1.0,0.000000,0.989899,0.989899,0.989899,0.919192,0.919192,0.919192,1.0,0.979798,1.000000,0.969697,1.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
2,0.111111,0.0,0.989899,0.010101,0.060606,0.181818,0.000000,0.020202,0.090909,1.0,0.505051,0.696970,0.000000,0.171717,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0,1
3,0.000000,0.0,0.929293,0.252525,0.505051,0.696970,0.010101,0.080808,0.171717,1.0,0.393939,0.454545,0.454545,0.222222,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0,1
4,1.000000,1.0,0.000000,0.010101,0.020202,0.161616,0.252525,0.585859,0.747475,1.0,0.060606,0.333333,0.090909,0.212121,1.0,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0


In [134]:
#save intermediate results:
final_data_y_pred.to_csv('final_data_y_pred.csv', index=False)

## Model Pipeline to check

In [133]:
#complete the code here
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier()
model2 = LogisticRegression()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

model_pipeline = [model1, model2]
model_names = ['Decision Tree Classifier', 'Logistic Regression']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

# We can use the result to choose the best performing model

{'Decision Tree Classifier': 0.8932646765520567, 'Logistic Regression': 0.9498617828554785}
